# 测试看一下GGCNN2
其实单从学习代码的角度来看，这块没必要写一个notebook，直接在py文件里面改就行了，但是这个东西我想结合着论文看一下，好好研究一下网络的思路和一些比较复杂网络的构建。

GG-CNN2 is a fully convolutional network based on the semantic segmentation architecture from Yu and Koltun (2016), which uses dilated convolutional layers to provide improved performance in semantic segmentation tasks.

GG-CNN2是一个基于语义分割模型（[Yu and Koltun（2016）](https://arxiv.org/abs/1511.07122)）的全卷积神经网络，它使用膨胀卷积层来提高语义分割任务的性能。而之前所使用的GGCNN仅仅是一个baseline而已。

GGCNN2和GGCNN有着同样的输入输出，只是采用了一些其他的测试来提升速度，跟一些直接从其他地方照搬过来进行语义分割的CNN相比，GGCNN和GGCNN2有着体量小，速度快的特点。网络结构调节的思路主要是在第8节，看了一下之后，发现调优工作主要是在jacquard数据集上来的，刚我也申请了这一数据集，正在下载，最晚后天能下载完吧。

另外读完了之后，我觉得这个也没有太多技巧可言，好像就是在试，边试边修剪，把那些发挥作用不大的都裁剪掉了，这样速度就上来了。

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GGCNN2(nn.Module):
    def __init__(self, input_channels=1, filter_sizes=None, l3_k_size=5, dilations=None):
        super().__init__()

        if filter_sizes is None:
            filter_sizes = [16,  # First set of convs
                            16,  # Second set of convs
                            32,  # Dilated convs
                            16]  # Transpose Convs

        if dilations is None:
            dilations = [2, 4]

        self.features = nn.Sequential(
            # 4 conv layers.
            nn.Conv2d(input_channels, filter_sizes[0], kernel_size=11, stride=1, padding=5, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(filter_sizes[0], filter_sizes[0], kernel_size=5, stride=1, padding=2, bias=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(filter_sizes[0], filter_sizes[1], kernel_size=5, stride=1, padding=2, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(filter_sizes[1], filter_sizes[1], kernel_size=5, stride=1, padding=2, bias=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Dilated convolutions.
            nn.Conv2d(filter_sizes[1], filter_sizes[2], kernel_size=l3_k_size, dilation=dilations[0], stride=1, padding=(l3_k_size//2 * dilations[0]), bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(filter_sizes[2], filter_sizes[2], kernel_size=l3_k_size, dilation=dilations[1], stride=1, padding=(l3_k_size//2 * dilations[1]), bias=True),
            nn.ReLU(inplace=True),

            # Output layers
            nn.ConvTranspose2d(filter_sizes[2], filter_sizes[3], 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(filter_sizes[3], filter_sizes[3], 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),

        )

        self.pos_output = nn.Conv2d(filter_sizes[3], 1, kernel_size=1)
        self.cos_output = nn.Conv2d(filter_sizes[3], 1, kernel_size=1)
        self.sin_output = nn.Conv2d(filter_sizes[3], 1, kernel_size=1)
        self.width_output = nn.Conv2d(filter_sizes[3], 1, kernel_size=1)

        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
                nn.init.xavier_uniform_(m.weight, gain=1)
    def forward(self, x):
        x = self.features(x)

        pos_output = self.pos_output(x)
        cos_output = self.cos_output(x)
        sin_output = self.sin_output(x)
        width_output = self.width_output(x)

        return pos_output, cos_output, sin_output, width_output

除了直观上看这个定义模型的算法更长了些之外，里面跟GGCNN至少有两点不同：
- 1.可以供更改的参数只有三个，一个是kernel size，一个是l3_k_size，还有一个就是dilation，其他参数都定死了，应该是测试后得到的比较好的结果。
- 2.这里把中间的运算层“打了个包”，定义成为了一个Squential序列模型，命名为特征提取层，这边会输出总的结果，最后再定义几个不同数据的反卷积输出层，这样做的好处一个是数据打包层次分明，模型大的时候不容易搞混，还有一个就是，可以看到，模型内的定义是不需要给每一层独立命名的，这就省了不少事，起码我觉得省了不少事

然后其他就没啥了，不出意外的话，我直接在原先的程序里面啥都不改，模型改调用这个，就是可以正常训练的。

## 注意
上面的代码是原始的GGCNN,也就是2020年5月份左右版本的代码,作者在2020年8月份更新了一次,将输出层改成了下面这样:

In [ ]:
# Output layers
nn.UpsamplingBilinear2d(scale_factor=2),
nn.Conv2d(filter_sizes[2], filter_sizes[3], 3, padding=1),
nn.ReLU(inplace=True),
nn.UpsamplingBilinear2d(scale_factor=2),
nn.Conv2d(filter_sizes[3], filter_sizes[3], 3, padding=1),
nn.ReLU(inplace=True),

其他都一样,这么一改提升了网络的精度和性能.

### 这是原来的网络结构:
```
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
================================================================
            Conv2d-1         [-1, 16, 300, 300]           7,760
              ReLU-2         [-1, 16, 300, 300]               0
            Conv2d-3         [-1, 16, 300, 300]           6,416
              ReLU-4         [-1, 16, 300, 300]               0
         MaxPool2d-5         [-1, 16, 150, 150]               0
            Conv2d-6         [-1, 16, 150, 150]           6,416
              ReLU-7         [-1, 16, 150, 150]               0
            Conv2d-8         [-1, 16, 150, 150]           6,416
              ReLU-9         [-1, 16, 150, 150]               0
        MaxPool2d-10           [-1, 16, 75, 75]               0
           Conv2d-11           [-1, 32, 75, 75]          12,832
             ReLU-12           [-1, 32, 75, 75]               0
           Conv2d-13           [-1, 32, 75, 75]          25,632
             ReLU-14           [-1, 32, 75, 75]               0
  ConvTranspose2d-15         [-1, 16, 150, 150]           4,624
             ReLU-16         [-1, 16, 150, 150]               0
  ConvTranspose2d-17         [-1, 16, 300, 300]           2,320
             ReLU-18         [-1, 16, 300, 300]               0
           Conv2d-19          [-1, 1, 300, 300]              17
           Conv2d-20          [-1, 1, 300, 300]              17
           Conv2d-21          [-1, 1, 300, 300]              17
           Conv2d-22          [-1, 1, 300, 300]              17
================================================================
Total params: 72,484
Trainable params: 72,484
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.37
Forward/backward pass size (MB): 94.07
Params size (MB): 0.28
Estimated Total Size (MB): 95.72
----------------------------------------------------------------
```
### 这是更新后的网络结构:
```
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
================================================================
            Conv2d-1         [-1, 16, 300, 300]           7,760
              ReLU-2         [-1, 16, 300, 300]               0
            Conv2d-3         [-1, 16, 300, 300]           6,416
              ReLU-4         [-1, 16, 300, 300]               0
         MaxPool2d-5         [-1, 16, 150, 150]               0
            Conv2d-6         [-1, 16, 150, 150]           6,416
              ReLU-7         [-1, 16, 150, 150]               0
            Conv2d-8         [-1, 16, 150, 150]           6,416
              ReLU-9         [-1, 16, 150, 150]               0
        MaxPool2d-10           [-1, 16, 75, 75]               0
           Conv2d-11           [-1, 32, 75, 75]          12,832
             ReLU-12           [-1, 32, 75, 75]               0
           Conv2d-13           [-1, 32, 75, 75]          25,632
             ReLU-14           [-1, 32, 75, 75]               0
UpsamplingBilinear2d-15      [-1, 32, 150, 150]               0
           Conv2d-16         [-1, 16, 150, 150]           4,624
             ReLU-17         [-1, 16, 150, 150]               0
UpsamplingBilinear2d-18      [-1, 16, 300, 300]               0
           Conv2d-19         [-1, 16, 300, 300]           2,320
             ReLU-20         [-1, 16, 300, 300]               0
           Conv2d-21          [-1, 1, 300, 300]              17
           Conv2d-22          [-1, 1, 300, 300]              17
           Conv2d-23          [-1, 1, 300, 300]              17
           Conv2d-24          [-1, 1, 300, 300]              17
================================================================
Total params: 72,484
Trainable params: 72,484
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.37
Forward/backward pass size (MB): 110.55
Params size (MB): 0.28
Estimated Total Size (MB): 112.20
----------------------------------------------------------------
```